In [1]:
from requests import Session
from tabulate import tabulate
import json
import io
session = Session()
categoryResponse = session.get('http://52.59.243.6:8000/listCategories/', headers={'Accept':'application/json'})
categories = json.loads(categoryResponse.text)
categoryList = [[category['id'], category['name'], [[subCategory['id'], subCategory['name'], subCategory['numOfProducts']] for subCategory in category['sub_categories']]] 
               for category in categories]
for categoryId, categoryName, subCategories in categoryList:
    print "----------------------------------------"
    print categoryName + " (ID: " + str(categoryId) +")"
    print tabulate(subCategories, headers=["Sub Id", "Name", "Num of Products"])

----------------------------------------
Beverages (ID: 8)
  Sub Id  Name             Num of Products
--------  -------------  -----------------
      19  Alcoholic                      0
      20  Non-alcoholic                  8
----------------------------------------
Dairy (ID: 10)
  Sub Id  Name              Num of Products
--------  --------------  -----------------
      21  Milk                            1
      22  Cultured Dairy                  3
      23  Cheese                          6
      24  Butter                          0
      25  Icecream                        2
      67  Eggs                            1
----------------------------------------
Ready-to-eat Food (ID: 11)
  Sub Id  Name              Num of Products
--------  --------------  -----------------
      15  Snack                          19
      16  Bakery                         19
      26  Pizza                           2
      27  Microwave food                  3
-----------------------------

In [4]:
#Request products from all sub category
allProducts = []
nTotalProducts = 0
for categoryId, categoryName, subCategories in categoryList:
    print "-------------------------------------------"
    productsResponse = session.get('http://52.59.243.6:8000/listProducts/?categoryId='+str(categoryId)+'&offset=0&len=200&expired_after=2015-01-01', 
                                   headers={'Accept':'application/json'})
    products = json.loads(productsResponse.text)
    print categoryName + "(ID=" + str(categoryId) + ")" + ": " + str(len(products))
    print products
    nTotalProducts += len(products)
    allProducts.append([categoryName, products])
    for subCategoryID, subCategoryName, _ in subCategories:
        productsResponse = session.get('http://52.59.243.6:8000/listProducts/?categoryId='+str(subCategoryID)+'&offset=0&len=200&expired_after=2015-01-01', 
                                   headers={'Accept':'application/json'})
        products = json.loads(productsResponse.text)
        print "--" + subCategoryName + "(ID=" + str(categoryId) + ")" + ": " + str(len(products))
        print products
        nTotalProducts += len(products)
        allProducts.append([subCategoryName, products])

print "Total Products: "  + str(nTotalProducts)

-------------------------------------------
Beverages(ID=8): 0
[]
--Alcoholic(ID=8): 0
[]
--Non-alcoholic(ID=8): 0
[]
-------------------------------------------
Dairy(ID=10): 0
[]
--Milk(ID=10): 4
[{u'description': u'1 liter', u'price': u'12.90', u'storeId': u'5', u'initial_price': u'None', u'from_date': u'2016-08-14', u'to_date': u'2016-08-21', u'image': None, u'discount_rate': u'None', u'id': 65, u'categoryId': u'21', u'name': u'Q skummet melk'}, {u'description': u'200 g. Pr kg 81,50', u'price': u'16.30', u'storeId': u'4', u'initial_price': u'None', u'from_date': u'2016-08-14', u'to_date': u'2016-08-28', u'image': None, u'discount_rate': u'None', u'id': 123, u'categoryId': u'21', u'name': u'Coop melkesjokolade'}, {u'description': u'Fra R\xf8rosmeieriet. 1 l. Fra 12,50/l. 13,00', u'price': u'25.00', u'storeId': u'3', u'initial_price': u'None', u'from_date': u'2016-08-28', u'to_date': u'2016-09-04', u'image': u'http://52.59.243.6:8000/media/product/de15YHFE.jpgm0', u'discount_rate': u

In [23]:
productWithImages = []
for categoryName, products in allProducts:
    filteredProducts = [product for product in products if product["image"]]
    productWithImages.append([categoryName, filteredProducts])
    print  '%22s : %d images' %(categoryName, len(filteredProducts))

             Beverages : 0 images
             Alcoholic : 0 images
         Non-alcoholic : 0 images
                 Dairy : 0 images
                  Milk : 2 images
        Cultured Dairy : 16 images
                Cheese : 37 images
                Butter : 0 images
              Icecream : 3 images
                  Eggs : 2 images
     Ready-to-eat Food : 0 images
                 Snack : 0 images
                Bakery : 0 images
                 Pizza : 7 images
        Microwave food : 5 images
  Fruit and Vegetables : 0 images
                 Fruit : 15 images
            Vegetables : 30 images
                  Meat : 0 images
                  Beef : 18 images
                  Pork : 12 images
      Chicken & Turkey : 16 images
                  Lamb : 0 images
              Reindeer : 0 images
                   Ham : 18 images
              Sausages : 7 images
              Meatball : 6 images
               Seafood : 0 images
                  Fish : 0 images
      

In [24]:
#Download all product images to the corresponding folder
import os
import urllib
import random

LEAST_EXAMPLES = 15
datasetDir = 'MatifyDataset/'
for categoryName, products in productWithImages:
    categoryPath =  os.path.join(datasetDir,
                              categoryName)
    if len(products) >= LEAST_EXAMPLES:
        if not os.path.exists(categoryPath):
            os.makedirs(categoryPath)
        for product in products:
            urllib.urlretrieve(product["image"], os.path.join(categoryPath, str(product["id"])+".jpg"))

In [60]:
def _get_filenames_and_classes(dataset_dir):
    """Returns a list of filenames and inferred class names.
    Args:
    dataset_dir: A directory containing a set of subdirectories representing
      class names. Each subdirectory should contain PNG or JPG encoded images.
    Returns:
    A list of image file paths, relative to `dataset_dir` and the list of
    subdirectories, representing class names.
    """
    root = dataset_dir
    directories = []
    class_names = []
    for filename in os.listdir(root):
        path = os.path.join(root, filename)
        if os.path.isdir(path):
            directories.append(path)
            class_names.append(filename)

    photo_filenames = []
    for directory in directories:
        for filename in os.listdir(directory):
            path = os.path.join(directory, filename)
            photo_filenames.append(path)

    return photo_filenames, sorted(class_names)

photo_filenames, class_names = _get_filenames_and_classes(datasetDir)
class_names_to_ids = dict(zip(class_names, range(len(class_names))))

# Divide into train and test:
# The number of images in the validation set.
_NUM_VALIDATION = 20

# Seed for repeatability.
_RANDOM_SEED = 0

random.seed(_RANDOM_SEED)
random.shuffle(photo_filenames)
training_filenames = photo_filenames[_NUM_VALIDATION:]
validation_filenames = photo_filenames[:_NUM_VALIDATION]


In [61]:
validation_filenames

['MatifyDataset/Cultured Dairy/264.jpg',
 'MatifyDataset/Fruit/440.jpg',
 'MatifyDataset/Cultured Dairy/305.jpg',
 'MatifyDataset/Processed products/434.jpg',
 'MatifyDataset/Beef/474.jpg',
 'MatifyDataset/Cultured Dairy/462.jpg',
 'MatifyDataset/Vegetables/495.jpg',
 'MatifyDataset/Vegetables/355.jpg',
 'MatifyDataset/Cultured Dairy/619.jpg',
 'MatifyDataset/Processed products/358.jpg',
 'MatifyDataset/Sauce & Dressing/326.jpg',
 'MatifyDataset/Cheese/217.jpg',
 'MatifyDataset/Processed products/592.jpg',
 'MatifyDataset/Ham/623.jpg',
 'MatifyDataset/Processed products/269.jpg',
 'MatifyDataset/Rice, Pasta and flour/116.jpg',
 'MatifyDataset/Processed products/400.jpg',
 'MatifyDataset/Fruit/596.jpg',
 'MatifyDataset/Chicken & Turkey/446.jpg',
 'MatifyDataset/Cheese/566.jpg']

In [62]:
import tensorflow as tf
import math
import sys
sys.path.append(os.path.abspath("/notebooks/AutoParser/squeezenet/models/slim/"))
from datasets import dataset_utils
_NUM_SHARDS = 1
class ImageReader(object):
    """Helper class that provides TensorFlow image coding utilities."""

    def __init__(self):
        # Initializes function that decodes RGB JPEG data.
        self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
        self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

    def read_image_dims(self, sess, image_data):
        image = self.decode_jpeg(sess, image_data)
        return image.shape[0], image.shape[1]

    def decode_jpeg(self, sess, image_data):
        image = sess.run(self._decode_jpeg,
                     feed_dict={self._decode_jpeg_data: image_data})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

def _get_dataset_filename(dataset_dir, split_name, shard_id):
    output_filename = 'matify_%s_%05d-of-%05d.tfrecord' % (
        split_name, shard_id, _NUM_SHARDS)
    return os.path.join(dataset_dir, output_filename)

def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir):
    """Converts the given filenames to a TFRecord dataset.
    Args:
    split_name: The name of the dataset, either 'train' or 'validation'.
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
    """
    assert split_name in ['train', 'validation']

    num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))

    with tf.Graph().as_default():
        image_reader = ImageReader()

        with tf.Session('') as sess:

            for shard_id in range(_NUM_SHARDS):
                output_filename = _get_dataset_filename(dataset_dir, split_name, shard_id)

                with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
                    start_ndx = shard_id * num_per_shard
                    end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
                    for i in range(start_ndx, end_ndx):
                        sys.stdout.write('\r>> Converting image %d/%d shard %d' % (i+1, len(filenames), shard_id))
                        sys.stdout.flush()

                        # Read the filename:
                        image_data = tf.gfile.FastGFile(filenames[i], 'r').read()
                        height, width = image_reader.read_image_dims(sess, image_data)

                        class_name = os.path.basename(os.path.dirname(filenames[i]))
                        class_id = class_names_to_ids[class_name]

                        example = dataset_utils.image_to_tfexample(
                            image_data, 'jpg', height, width, class_id)
                        tfrecord_writer.write(example.SerializeToString())

    sys.stdout.write('\n')
    sys.stdout.flush()


In [63]:
# First, convert the training and validation sets.
_convert_dataset('train', training_filenames, class_names_to_ids,
               datasetDir)
_convert_dataset('validation', validation_filenames, class_names_to_ids,
               datasetDir)

# Finally, write the labels file:
labels_to_class_names = dict(zip(range(len(class_names)), class_names))
dataset_utils.write_label_file(labels_to_class_names, datasetDir)

print('\nFinished converting the Matify dataset!')


>> Converting image 221/221 shard 0
>> Converting image 20/20 shard 0

Finished converting the Matify dataset!
